In [29]:
import os, os.path
import numpy as np
import pandas as pd
import model_attributes as ma
from attribute_table import AttributeTable
import setup_analysis as sa
import support_classes as sc
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
import geopandas as gpd
import rioxarray as rx
import itertools
import model_afolu as mafl

In [30]:
dir_data = "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/AFOLU"

# set names 
fn_climates = "kc_1984_2013.tif"
fn_countries = "WB_countries_Admin0_10m"
fn_cw = "values_info_with_cw_kc_1984_2013.csv"

fp_climates = os.path.join(dir_data, fn_climates)
fp_countries = os.path.join(dir_data, fn_countries, f"{fn_countries}.shp")
fp_cw = os.path.join(dir_data, fn_cw)

model_afolu = mafl.AFOLU(sa.model_attributes)
regions = sc.Regions(sa.model_attributes)
time_periods = sc.TimePeriods(sa.model_attributes)

/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_attributes.py:2309: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  cols = list(set(attr_table.table.columns & set(fields_to_filter_on)))
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_attributes.py:2309: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  cols = list(set(attr_table.table.columns & set(fields_to_filter_on)))
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_attributes.py:2309: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  cols = list(set(a

In [3]:
# convert geotiff to dataframe
rx_array = rx.open_rasterio(fp_climates)
df_climates = rx_array[0].to_pandas()
# retrieve climate categories
df_climate_cats = pd.read_csv(fp_cw, sep = "|")

In [23]:

def reshape_df_climate(
    df_input: pd.DataFrame,
    vec_x: np.ndarray,
    vec_y: np.ndarray,
    field_val: str = "value",
    field_x: str = "x",
    field_y: str = "y",
    flag_empty: int = flag_empty,
) -> pd.DataFrame:
    """
    Reshape df soils without having to melt and filter
    """
    arr = np.array(df_input)

    w = np.where(arr != flag_empty)
    
    # get indexing
    col_vals = arr[w[0], w[1]]
    col_x = vec_x[w[1]]
    col_y = vec_y[w[0]]
    
    df_out = pd.DataFrame(
        {
            field_x: col_x,
            field_y: col_y,
            field_val: col_vals
        }
    )
    
    return df_out



flag_empty = df_climates.iloc[0,0]
vec_x = np.array(df_climates.columns)
vec_y = np.array(df_climates.index)


dfc = reshape_df_climate(
    df_climates,
    vec_x,
    vec_y,
    flag_empty = flag_empty
)

crs_climates = "EPSG:4326"
gdf_climates_cut = dfc.copy()

In [27]:
#flag_empty = -3.4e+38
#np.where(np.array(df_climates.iloc[0:5, 0:5]) == )




In [47]:
"""
df_climates = df_climates.reset_index()#[-179.97083333333333].unique()
dfc = pd.melt(df_climates, ["y"]);
val = float(dfc["value"].iloc[0])
dfc = dfc[dfc["value"] != val].reset_index(drop = True)

# set some bounds
# http://bboxfinder.com/#-55.776573,-126.386719,33.431441,-33.398438
x_max = -34
x_min = -119
y_max = 33
y_min = -58
# filter out
dfc2 = dfc[
    (dfc["x"] <= x_max) & 
    (dfc["x"] >= x_min) & 
    (dfc["y"] <= y_max) & 
    (dfc["y"] >= y_min)
].reset_index(drop = True)

# create points with climates
gdf_climates = gpd.GeoDataFrame(
    dfc2, 
    geometry = gpd.points_from_xy(dfc2["x"], dfc2["y"])
)
gdf_climates.crs = "EPSG:4326"
gdf_climates_cut = gdf_climates.copy()
""";

In [31]:
# get countries 
countries_keep = regions.all_isos#list(sa.model_attributes.dict_attributes.get("region").table["category_name"])
gdf_world = gpd.read_file(fp_countries)

# some replacements
field_en = "ISO_A3"
gdf_world_red = gdf_world[gdf_world[field_en].isin(countries_keep)]
gdf_lac = gdf_world_red[[field_en, "geometry"]]
gdf_lac_bounds = gdf_lac.bounds



In [40]:
gdf_points = gdf_climates_cut[
    (gdf_climates_cut["x"] <= maxx) &
    (gdf_climates_cut["x"] >= minx) &
    (gdf_climates_cut["y"] <= maxy) &
    (gdf_climates_cut["y"] >= miny)
]



In [66]:
def df_to_gdf(
    df_in: pd.DataFrame,
    crs: str = crs_climates,
    field_x: str = "x",
    field_y: str = "y",
) -> gpd.GeoDataFrame:
    
    vec_x = np.array(df_in[field_x])
    vec_y = np.array(df_in[field_y])
    
    gdf_out = gpd.GeoDataFrame(
        df_in, 
        geometry = [shapely.geometry.Point(*x) for x in zip(vec_x, vec_y)]
    )
    gdf_out.crs = crs
    
    return gdf_out
    
t0 = time.time()
gdf_points2 = df_to_gdf(gdf_points)
sf.get_time_elapsed(t0)

58.58

In [61]:
#sf.get_time_elapsed(t0)
import shapely
type(gdf_points2.iloc[0]["geometry"])

vec_x = np.array(gdf_points["x"])[0:100000]
vec_y = np.array(gdf_points["y"])[0:100000]

t0 = time.time()
xx = [shapely.geometry.Point(*x) for x in zip(vec_x, vec_y)]
print(sf.get_time_elapsed(t0))

t0 = time.time()
xx2 = gpd.points_from_xy(vec_x, vec_y, crs = crs_climates)
print(sf.get_time_elapsed(t0))


0.68
2.08


In [67]:

i = 0
sep = "---"*25
print("Copying climate gdf...\n\n")
#gdf_climates_cut = gdf_climates.copy()


df_out = []
df_agg_out = []
t0 = time.time()
list_times = [t0]
n = len(gdf_lac_bounds)

while i < n:
    
    # country
    gdf_country = gdf_lac.iloc[i:(i + 1)]
    bounds = gdf_lac_bounds.iloc[i:(i + 1)]

    country = str(gdf_country[field_en].iloc[0])
    maxx = float(bounds["maxx"].iloc[0])
    minx = float(bounds["minx"].iloc[0])
    maxy = float(bounds["maxy"].iloc[0])
    miny = float(bounds["miny"].iloc[0])
    
    pos = i + 1
    print(f"Starting country {country}\n{sep} ({pos}/{n})\n")
    
    # get points
    print("Filtering points...\n")
    gdf_points = gdf_climates_cut[
        (gdf_climates_cut["x"] <= maxx) &
        (gdf_climates_cut["x"] >= minx) &
        (gdf_climates_cut["y"] <= maxy) &
        (gdf_climates_cut["y"] >= miny)
    ]
    
    gdf_points = gpd.GeoDataFrame(
        gdf_points,
        geometry = gpd.points_from_xy(gdf_points["x"], gdf_points["y"], crs = crs_climates)
    )
    
    print(f"Merging points within bbox x = ({minx}, {maxx}), y = ({miny}, {maxy}) to country...\n")
    gdf_join = gpd.sjoin(
        gdf_country, 
        gdf_points[["value", "geometry"]].rename(columns = {"value": "kcc"}),
        how = "inner"
    ).drop("geometry", axis = 1)
    
    
    print(f"Aggregating counts by climate...\n")
    gdf_agg = gdf_join[[field_en, "kcc"]].copy()
    gdf_agg["count"] = 1
    gdf_agg = (
        gdf_agg[[field_en, "kcc", "count"]]
        .groupby([field_en, "kcc"])
        .agg(
            {field_en: "first", "kcc": "first", "count": "sum"}
        )
        .reset_index(drop = True)
    )

    
    print("Appending output...")
    df_out.append(gdf_join)
    df_agg_out.append(gdf_agg)
    
    
    print("Dropping indices...")
    indices_to_drop = list(gdf_join["index_right"])
    gdf_climates_cut.drop(indices_to_drop, axis = 0, inplace = True)
    
    
    i += 1
    list_times.append(time.time())
    t_delta = np.round(list_times[i] - list_times[i - 1], 2)
    t_elapsed = np.round(list_times[i] - list_times[0])
    
    print(f"\n{sep}\n\nCountry {country} complete in {t_delta} seconds ({t_elapsed} seconds overall)\n\n{sep}\n\n\n")
    

Copying climate gdf...


Starting country IDN
--------------------------------------------------------------------------- (1/216)

Filtering points...

Merging points within bbox x = (95.01270592493853, 140.97762699419047), y = (-10.92262135219363, 5.910101630386009) to country...

Aggregating counts by climate...

Appending output...
Dropping indices...

---------------------------------------------------------------------------

Country IDN complete in 155.7 seconds (156.0 seconds overall)

---------------------------------------------------------------------------



Starting country MYS
--------------------------------------------------------------------------- (2/216)

Filtering points...

Merging points within bbox x = (99.64522806022688, 119.2780867853769), y = (0.8513703412683071, 7.355780340990009) to country...

Aggregating counts by climate...

Appending output...
Dropping indices...

---------------------------------------------------------------------------

Country MYS co

# NOTE: CAN START HERE
- Only need to unzip and load `os.path.join(dir_data, "kcc_cells_merged_to_country", "kcc_coords_index.csv")`
- country-level values are stored in `os.path.join(dir_data, "kcc_cells_merged_to_country")` with name `f"kcc_by_country_{iso}.csv"`

In [83]:
pd.DataFrame(
    dfc
).to_csv(
    os.path.join(dir_data, "kcc_cells_merged_to_country", "kcc_coords_index.csv"),
    index = None,
    encoding = "UTF-8"
)

In [70]:
#df_out_full = pd.concat(df_out, axis = 0).reset_index(drop = True);
df_out_agg = pd.concat(df_agg_out, axis = 0).reset_index(drop = True);

# export output
#df_out_full.to_csv(sa.fp_csv_kcc_cells_merged_to_country, index = None, encoding = "UTF-8")
df_out_agg.to_csv(sa.fp_csv_kcc_cell_counts_by_country_kcc, index = None, encoding = "UTF-8")

In [75]:
dir_exp = os.path.join(dir_data, "kcc_cells_merged_to_country")
os.makedirs(dir_exp, exist_ok = True)

for df in df_out:
    iso = str(df[field_en].iloc[0])
    fp_write = os.path.join(dir_exp, f"kcc_by_country_{iso}.csv")
    df.drop(field_en, axis = 1).to_csv(fp_write, index = None, encoding = "UTF-8")
    print(f"country '{iso}' complete.\n")
    

country 'IDN' complete.

country 'MYS' complete.

country 'CHL' complete.

country 'BOL' complete.

country 'PER' complete.

country 'ARG' complete.

country 'CYP' complete.

country 'IND' complete.

country 'CHN' complete.

country 'ISR' complete.

country 'LBN' complete.

country 'ETH' complete.

country 'SSD' complete.

country 'SOM' complete.

country 'KEN' complete.

country 'PAK' complete.

country 'MWI' complete.

country 'TZA' complete.

country 'SYR' complete.

country 'SUR' complete.

country 'GUY' complete.

country 'KOR' complete.

country 'PRK' complete.

country 'MAR' complete.

country 'CRI' complete.

country 'NIC' complete.

country 'COG' complete.

country 'COD' complete.

country 'BTN' complete.

country 'UKR' complete.

country 'BLR' complete.

country 'NAM' complete.

country 'ZAF' complete.

country 'MAF' complete.

country 'SXM' complete.

country 'OMN' complete.

country 'UZB' complete.

country 'KAZ' complete.

country 'TJK' complete.

country 'LTU' complete.



In [96]:


##
##  BUILD FRACTIONS WET/DRY AND TEMPERATE/TROPICAL BY COUNTRY
##

df_out_agg["count"] = df_out_agg["count"].astype(int)

# get total counts
df_out_agg_totals = (
    sf.simple_df_agg(
        df_out_agg[[field_en, "count"]],
        [field_en],
        {"count": "sum"}
    )
    .rename(columns = {"count": "total_count"})
)
# 
df_climate_by_country = pd.merge(
    df_out_agg, 
    df_climate_cats[["code_num", "wet_dry_cat", "temperate_tropical_cat"]].rename(columns = {"code_num": "kcc"})
)

# break out by wet/dry and temperate/tropical
df_climate_by_country_temptrop = df_climate_by_country.drop(["wet_dry_cat"], axis = 1)
df_climate_by_country_wetdry = df_climate_by_country.drop(["temperate_tropical_cat"], axis = 1)
all_countries = sorted(list(set(df_climate_by_country[field_en])))

# loop over different splits
fields_loop = ["wet_dry_cat", "temperate_tropical_cat"]
dict_df_climate_aggs = {}


if True:
    for field in fields_loop:

        fields_grp = [field_en, field]
        fields_dat = ["count"]

        all_vals = sorted(list(set(df_climate_by_country[field])))
        df_tmp = df_climate_by_country.drop([x for x in fields_loop if (x != field)], axis = 1)
        dict_agg = dict(zip(fields_grp, ["first" for x in fields_grp]))
        dict_agg.update(dict(zip(fields_dat, ["sum" for x in fields_dat])))

        df_tmp = df_tmp[fields_grp + fields_dat].groupby(fields_grp).agg(dict_agg).reset_index(drop = True)
        df_expand = pd.DataFrame(itertools.product(all_countries, all_vals), columns = [field_en, field])
        df_expand = pd.merge(df_expand, df_out_agg_totals, how = "left").fillna(0)

        df_tmp = pd.merge(df_expand, df_tmp, how = "left").fillna(0)
        df_tmp["frac"] = np.array(df_tmp["count"])/np.array(df_tmp["total_count"])
        df_tmp.rename(columns = {field_en: "country"}, inplace = True)


        # get pivot
        df_piv = pd.pivot(
            df_tmp[["country", field, "frac"]],
            index = ["country"],
            columns = [field], 
            values = ["frac"]
        ).reset_index()
        df_piv.columns = df_piv.columns.to_flat_index()
        dict_rnm = dict([(x, x[1].replace(")", "").strip()) for x in df_piv.columns if (x[1] != "")])
        dict_rnm.update({("country", ""): "country"})
        df_piv.rename(columns = dict_rnm, inplace = True)

        dict_df_climate_aggs.update({field: df_piv})


In [121]:
##  TEMPERATE/TROPICAL SPLITS

varlist_tt = []
# ag temp/trop vars
varlist_tt += sa.model_attributes.build_varlist(sa.model_attributes.subsec_name_agrc, model_afolu.modvar_agrc_frac_temperate)
varlist_tt += sa.model_attributes.build_varlist(sa.model_attributes.subsec_name_agrc, model_afolu.modvar_agrc_frac_tropical)
# frst temp/trop vars
varlist_tt += sa.model_attributes.build_varlist(sa.model_attributes.subsec_name_frst, model_afolu.modvar_frst_frac_temperate_nutrient_rich)
varlist_tt += sa.model_attributes.build_varlist(sa.model_attributes.subsec_name_frst, model_afolu.modvar_frst_frac_temperate_nutrient_poor)
varlist_tt += sa.model_attributes.build_varlist(sa.model_attributes.subsec_name_frst, model_afolu.modvar_frst_frac_tropical)
# lndu temp/trop vars
varlist_tt += sa.model_attributes.build_varlist(sa.model_attributes.subsec_name_lndu, model_afolu.modvar_lndu_frac_temperate)
varlist_tt += sa.model_attributes.build_varlist(sa.model_attributes.subsec_name_lndu, model_afolu.modvar_lndu_frac_tropical)


##  WET/DRY SPLITS

varlist_wd = []
# ag temp/trop vars
varlist_wd += sa.model_attributes.build_varlist(sa.model_attributes.subsec_name_agrc, model_afolu.modvar_agrc_frac_wet)
varlist_wd += sa.model_attributes.build_varlist(sa.model_attributes.subsec_name_agrc, model_afolu.modvar_agrc_frac_dry)
# lndu temp/trop vars
varlist_wd += sa.model_attributes.build_varlist(sa.model_attributes.subsec_name_lndu, model_afolu.modvar_lndu_frac_wet)
varlist_wd += sa.model_attributes.build_varlist(sa.model_attributes.subsec_name_lndu, model_afolu.modvar_lndu_frac_dry)



In [138]:

df_years = pd.DataFrame({time_periods.field_year: range(2010, time_periods.year_max + 1)})

df_ret = dict_df_climate_aggs[field][["country"]].copy()

for field in dict_df_climate_aggs.keys():
    
    loop = varlist_tt if (field == "temperate_tropical_cat") else varlist_wd
    for var in loop:
        if field == "temperate_tropical_cat":
            fld = "temperate" if ("temperate" in var) else ("tropical")
        else:
            fld = "wet" if ("wet" in var) else "dry"
        div = 2 if (("nutrient_poor" in var) or ("nutrient_rich" in var)) else 1
        df_merge = dict_df_climate_aggs[field][["country", fld]].copy().rename(columns = {fld: var})
        df_ret = pd.merge(df_ret, df_merge)
        df_ret[var] = np.array(df_ret[var])/div

# merge in time period
df_ret = sf.explode_merge(
    df_years,
    df_ret
)
df_ret = time_periods.years_to_tps(df_ret)
df_ret.rename(columns = {"country": regions.field_iso}, inplace = True)
fields_sort = [regions.field_iso, time_periods.field_time_period, time_periods.field_year]

df_ret = (
    df_ret[fields_sort + sorted([x for x in df_ret.columns if (x not in fields_sort)])]
    .sort_values(by = fields_sort)
    .reset_index(drop = True)
)

df_ret.to_csv(
    sa.fp_csv_climate_fields_by_country_simple, index = None, encoding = "UTF-8"
)

In [140]:
#[[fields_sort] + sorted([x for x in df_ret.columns if (x not in fields_sort)])]


In [123]:


for var_name in varlist_soc:
    df_agg_all_fields[var_name] = np.array(df_agg_all_fields[field_cp])

df_agg_all_fields.drop(field_cp, axis = 1, inplace = True)
df_agg_all_fields.rename(columns = {field_en: field_country}, inplace = True)
    




country  time_period  frac_agrc_bevs_and_spices_cl1_temperate  \
0        ABW            0                                 1.000000   
1        ABW            1                                 1.000000   
2        ABW            2                                 1.000000   
3        ABW            3                                 1.000000   
4        ABW            4                                 1.000000   
...      ...          ...                                      ...   
7627     ZWE           31                                 0.989726   
7628     ZWE           32                                 0.989726   
7629     ZWE           33                                 0.989726   
7630     ZWE           34                                 0.989726   
7631     ZWE           35                                 0.989726   

      frac_agrc_bevs_and_spices_cl1_tropical  \
0                                   0.000000   
1                                   0.000000   
2                                   0.000000   
3                                   0.000000   
4                                   0.000000   
...                                      ...   
7627                                0.010274   
7628                                0.010274   
7629                                0.010274   
7630                                0.010274   
7631                                0.010274   

      frac_agrc_bevs_and_spices_cl2_dry  frac_agrc_bevs_and_spices_cl2_wet  \
0                              1.000000                           0.000000   
1                              1.000000                           0.000000   
2                              1.000000                           0.000000   
3                              1.000000                           0.000000   
4                              1.000000                           0.000000   
...                                 ...                                ...   
7627                           0.989726                           0.010274   
7628                           0.989726                           0.010274   
7629                           0.989726                           0.010274   
7630                           0.989726                           0.010274   
7631                           0.989726                           0.010274   

      frac_agrc_cereals_cl1_temperate  frac_agrc_cereals_cl1_tropical  \
0                            1.000000                        0.000000   
1                            1.000000                        0.000000   
2                            1.000000                        0.000000   
3                            1.000000                        0.000000   
4                            1.000000                        0.000000   
...                               ...                             ...   
7627                         0.989726                        0.010274   
7628                         0.989726                        0.010274   
7629                         0.989726                        0.010274   
7630                         0.989726                        0.010274   
7631                         0.989726                        0.010274   

      frac_agrc_cereals_cl2_dry  frac_agrc_cereals_cl2_wet  ...  \
0                      1.000000                   0.000000  ...   
1                      1.000000                   0.000000  ...   
2                      1.000000                   0.000000  ...   
3                      1.000000                   0.000000  ...   
4                      1.000000                   0.000000  ...   
...                         ...                        ...  ...   
7627                   0.989726                   0.010274  ...   
7628                   0.989726                   0.010274  ...   
7629                   0.989726                   0.010274  ...   
7630                   0.989726                   0.010274  ...   
7631                   0.989726     